<a href="https://colab.research.google.com/github/paulgureghian/Keras_to_TPU/blob/master/Keras_on_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Created by Paul A. Gureghian in April 2019.**


### **This scientific notebook contains Python code on how to train an LSTM Model using Keras and Google CoLaboratory with TPUs to exponentially reduce training time compared to a GPU on a local machine.**

### **An overview of the workflow.**

* Build a Keras model for training in functional API with static input batch_size.
* Convert Keras model to TPU model.
* Train the TPU model with static batch_size * 8 and save the weights to file.
* Build a Keras model for inference with the same structure but variable batch input size.
* Load the model weights.
* Predict with the inferencing model.

In [0]:
### Import packages
import os
import time
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding

In [11]:
### Define some training parameters, and download the dataset

# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words (among top max_features most common words)
maxlen = 500

# Load data
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print('IMDB Downloaded')

# Reverse sequences
x_train = [x[::-1] for x in x_train]
x_test = [x[::-1] for x in x_test]

# Pad sequences
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

IMDB Downloaded


In [0]:
### Define 'make_model' function

def make_model(batch_size=None):
    source = Input(shape=(maxlen,), batch_size=batch_size,
                   dtype=tf.int32, name='Input')
    embedding = Embedding(input_dim=max_features,
                          output_dim=128, name='Embedding')(source)
    lstm = LSTM(32, name='LSTM')(embedding)
    predicted_var = Dense(1, activation='sigmoid', name='Output')(lstm)
    model = tf.keras.Model(inputs=[source], outputs=[predicted_var])
    model.compile(
        optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
        loss='binary_crossentropy',
        metrics=['acc'])
    return model

In [13]:
### Print Keras model summary

tf.keras.backend.clear_session()
training_model = make_model(batch_size = 128)
training_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (128, 500)                0         
_________________________________________________________________
Embedding (Embedding)        (128, 500, 128)           1280000   
_________________________________________________________________
LSTM (LSTM)                  (128, 32)                 20608     
_________________________________________________________________
Output (Dense)               (128, 1)                  33        
Total params: 1,300,641
Trainable params: 1,300,641
Non-trainable params: 0
_________________________________________________________________


### **Convert Keras model to TPU model.**

In [14]:
### Use a builtin TensorFlow function to convert Keras to TPU

# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

INFO:tensorflow:Querying Tensorflow master (grpc://10.116.189.10:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 10432534271108538128)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17569796570915582283)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4595398824277185729)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5257321798323247484)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 1773730139355646359)
INFO:tensorflow:*** Available Device: _DeviceAttributes

In [15]:
### Print TPU model summary

tpu_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (128, 500)                0         
_________________________________________________________________
Embedding (Embedding)        (128, 500, 128)           1280000   
_________________________________________________________________
LSTM (LSTM)                  (128, 32)                 20608     
_________________________________________________________________
Output (Dense)               (128, 1)                  33        
Total params: 1,300,641
Trainable params: 1,300,641
Non-trainable params: 0
_________________________________________________________________


In [16]:
### Train the TPU model

start_time = time.time()

history = tpu_model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128 * 8,
                    validation_split=0.2)
tpu_model.save_weights('./tpu_model.h5', overwrite=True)

print("--- %s seconds ---" % (time.time() - start_time))

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(128,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(128, 500), dtype=tf.int32, name='Input_10'), TensorSpec(shape=(128, 1), dtype=tf.float32, name='Output_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 3.7073402404785156 secs
INFO:tensorflow:Setting weights on TPU model.
23552/25000 [===========================>..] - ETA: 0s - loss: 0.6931 - acc: 0.5028INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(53,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(53, 500), dtype=tf.int32, name='Input_10'), TensorSpec(shape=(53, 1), dtype=tf.float32, name='Output_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Rema

In [17]:
### Evaluluate the model

tpu_model.evaluate(x_test, y_test, batch_size=128 * 8)

ValueError: ignored